### Part 1 - Import de todas as bibliotecas necessárias 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Part 2 - Leitura do local dataframe + infos básicas

In [ ]:
df = pd.read_csv('RTA Dataset.csv')

In [ ]:
df.isnull().sum()   #verificar quantas colunas têm valores nulos

### Part 3 - Analisar e eliminar colunas não importantes

Nesta parte, analisamos o que significam algumas colunas que aparentam não ser tão importantes para a nossa análise

Após a análise, eliminamos as colunas que afetam pouco a nossa saída, que, como sabemos, é a última coluna: "Gravidade do Acidente"

In [ ]:
colunas_a_eliminar = ['Type_of_vehicle', 'Owner_of_vehicle', 'Defect_of_vehicle', 'Area_accident_occured',
                     'Pedestrian_movement', 'Fitness_of_casuality', 'Work_of_casuality', 'Time', 'Day_of_week']

df.drop(colunas_a_eliminar, axis = 1, inplace = True)

In [ ]:
df

Agora, para uma melhor compreensão da tabela, vamos renomear as colunas

In [ ]:
#time, day_of_week, age_band_of_driver, sex_of_driver, educational_level, vehicle_driver_relation, driving_experience
#type_of_vehicle, owner_of_vehicle, service_year_of_vehicle, defect_of_vehicle, area_accident_occured
#lanes_or_medians, road_allignment, types_of_junction, road_surface_type, road_surface_conditions, ligh_conditions
#weather_conditions, type_of_collision, number_of_vehicles_involved, number_of_casualties, vehicle_movement
#casualty_class, sex_of_casualty, age_band_of_casualty, casualty_severity, work_of_casualty, fitness_of_casualty
#pedestrian_movement, cause_of_accident, accident_severity,
colunas_renomeadas = ['Faixa Etária', 'Género', 'Nível de Educação', 'Relação com o Veículo', 
                      'Experiência de Condução', 'Idade do Veículo', 'Situação de Faixa', 'Alinhamento da Estrada', 'Tipo de Cruzamento',
                      'Tipo de Estrada', 'Condições do Piso', 'Condições de Visibilidade',
                      'Condiçoes Meteorológicas', 'Tipo de Colisão', 'N.º Veículos Envolvidos',
                      'Número de Vítimas', 'Movimento do Veículo', 'Tipo de Vítima','Género da Vítima',
                      'Faixa Etária da Vítima', 'Gravidade da Vítima',
                      'Causa do Acidente', 'Gravidade do Acidente']

df.columns = colunas_renomeadas
df

### Part 4 - Análise e substituição de valores nulos

Nesta parte, substituímos os valores escritos "na" por um valor verdadeirmente nulo, "NaN"

In [ ]:
df.replace('na', pd.NA, inplace=True)
df.isnull().sum()

Aqui percebemos a proporção de cada valor em cada coluna, incluindo os valores nulos

In [ ]:
percentage_values = {}
for column in df.columns:
    counts = df[column].value_counts(normalize=True, dropna=False) * 100
    percentage_values[column] = counts

for column, percentages in percentage_values.items():
    print(percentages)
    print()

Vamos ainda realizar a substituição de valores nulos pela moda da coluna.

Neste passo, fomos apenar substituir os valores nulos cujas colunas tinham menos de 1000 valores "NaN", uma vez que afeta pouco a proporção dos valores

In [ ]:
df['Nível de Educação'] = df['Nível de Educação'].fillna(df['Nível de Educação'].mode()[0])
df['Relação com o Veículo'] = df['Relação com o Veículo'].fillna(df['Relação com o Veículo'].mode()[0])
df['Experiência de Condução'] = df['Experiência de Condução'].fillna(df['Experiência de Condução'].mode()[0])
df['Situação de Faixa'] = df['Situação de Faixa'].fillna(df['Situação de Faixa'].mode()[0])
df['Tipo de Cruzamento'] = df['Tipo de Cruzamento'].fillna(df['Tipo de Cruzamento'].mode()[0])
df['Tipo de Estrada'] = df['Tipo de Estrada'].fillna(df['Tipo de Estrada'].mode()[0])
df['Tipo de Colisão'] = df['Tipo de Colisão'].fillna(df['Tipo de Colisão'].mode()[0])
df['Movimento do Veículo'] = df['Movimento do Veículo'].fillna(df['Movimento do Veículo'].mode()[0])

#substituir por valores em que a coluna da gravidade é a mesma

In [ ]:
df.isnull().sum()   #verificar quantas colunas têm valores nulos

In [ ]:
#proporções novas de todas as colunas
percentage_values = {}
for column in df.columns:
    counts = df[column].value_counts(normalize=True, dropna=False) * 100
    percentage_values[column] = counts

for column, percentages in percentage_values.items():
    print(percentages)
    print()

# ELIMINAR AS COLUNAS 4000 E TRAZER ALGUMAS DE VOLTA QUE ELIMINÁMOS MAIS CEDO

# várias experiências (exp1 - manter proporção, exp2 - subs pela moda, exp3 - fazer o KNN, exp4 - eliminar colunas)

Ao analisar as 4 colunas com mais de 1000 valores "NA", temos 2 soluções: ou eliminamos as colunas, ou repartimos a sua proporção pelos restantes valores da coluna.

Eliminar uma coluna

In [ ]:
colunas_a_eliminar_2 = ['Género da Vítima']
df.drop(colunas_a_eliminar_2, axis = 1, inplace = True)

Repartir a proporção pelos restantes valores das colunas

In [ ]:
# Define the columns to process
columns_to_process = ['Gravidade da Vítima', 'Faixa Etária da Vítima', 'Ocupação da Vítima']

for column in columns_to_process:
    # Number of null values in the column
    null_count = df[column].isnull().sum()

    # Proportion of non-null values for each category
    category_proportions = df[column].value_counts(normalize=True)

    # Calculate the number of null values to be distributed for each category
    null_distribution = (category_proportions * null_count).round().astype(int)

    # Randomly assign the null values to each category
    null_indices = df[df[column].isnull()].index
    for category, count in null_distribution.items():
        sample_indices = np.random.choice(null_indices, size=count, replace=False)
        df.loc[sample_indices, column] = category

    # Fill any remaining null values randomly
    remaining_null_indices = df[df[column].isnull()].index
    remaining_categories = list(category_proportions.index)
    for index in remaining_null_indices:
        df.at[index, column] = np.random.choice(remaining_categories)

Num ponto à parte, percebemos que eliminar as linhas com valores nulos não é uma opção, uma vez que perderíamos metade dos dados 

In [ ]:
no_linhas_na = df.isna().any(axis=1).sum()

print("Total de linhas com valores NaN", no_linhas_na)

### Part 5 - Fazer matriz de correlação e importância de variáveis

Primeiramente, vamos analisar a matriz de correlação entre as várias variáveis

In [ ]:
label_encoder = LabelEncoder()  # aplicar Label Encoder a todas as colunas "object"
for column in df.select_dtypes(include = 'object').columns:
    df[column] = label_encoder.fit_transform(df[column])

correlation_matrix = df.corr()
plt.figure(figsize = (20, 15))
sns.heatmap(correlation_matrix, annot = True, cmap = 'vlag', fmt = ".2f", linewidths = 1, square=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df_2 = df[['Gravidade da Vítima', 'Gravidade do Acidente']]
correlation_matrix = df_2.corr()
plt.figure(figsize = (10, 5))
sns.heatmap(correlation_matrix, annot = True, cmap = 'vlag', fmt = ".4f", linewidths = 1, square=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
contingency_table = pd.crosstab(df['Gravidade do Acidente'], df['Gravidade da Vítima'], normalize='index') * 100

# Display the proportions
print(contingency_table)

De seguida, analisamos a importância de cada feature para o modelo 

In [ ]:
X = df.iloc[:, :-1].values # todas as features
Y =  df.iloc[:, -1].values # feature target

modelo = ExtraTreesClassifier()
modelo.fit(X, Y)

feature_importances = pd.Series(modelo.feature_importances_, index = df.columns[:-1])
feature_importances.nlargest(10).plot(kind='barh')  # mostrar as 10 features mais importantes

plt.show()

Neste ponto, dividimos os dados em sets de treino e de teste, utilizando 25% dos dados para teste e os restantes para teino

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 30)

print("Número de exemplos nos dados de treino: ", X_train.shape[0])
print("Número de exemplos nos dados de teste: ", X_test.shape[0])

### Parte 6 - Árvores de Decisão

O primeiro modelo de machine learning utilizado foi o modelo de árvores de decisão.


Primeiro vamos estanderizar as nossas features. Isso é feito para garantir que todas as características tenham a mesma escala, o que pode melhorar o desempenho do algoritmo. 

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# O parâmetro max_depth = 4 define a profundidade máxima da árvore de decisão, 
# visto que limitar a profundidade pode ajudar a evitar overfitting, tornando a árvore mais simples.
clf = DecisionTreeClassifier(random_state = 42, criterion = "entropy", max_depth = 4)

# Treino: Arvore de Decisão
clf = clf.fit(X_train, y_train)

# Previsão da resposta --> Teste
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

# Print dos valores accuracy
print('Train data accuracy: ', accuracy_score(y_true = y_train, y_pred = y_train_pred))
print('Test data accuracy: ', accuracy_score(y_true = y_test, y_pred = y_test_pred))
print('Decision tree accuracy: ', accuracy_score(y_test_pred, y_test))



In [ ]:
plt.figure(figsize = (20, 10))
tree.plot_tree(clf, fontsize = 8, feature_names = df.columns[:-1], filled = True, rounded = True, proportion=True)
plt.show()

Através do classification report, podemos concluir a acurácia do modelo tal como o recall, a precisão e o f1-score.

In [ ]:
print(classification_report(y_test, y_test_pred, zero_division = 1))

Após realizar um classification report, vamos agora mostrar a matriz de confusão. Assim podemos ver a quantidade de falsos positivos e falsos negativos.

In [ ]:
cf_matrix = confusion_matrix(y_test, y_test_pred)

ax = sns.heatmap(cf_matrix, annot = True, cmap = 'seismic', fmt = "4.0f")
ax.set_title('Seaborn Confusion Matrix\n\n')
ax.set_xlabel('\nPredicted decision case disposition')
ax.set_ylabel('Actual decision case disposition ')

print(cf_matrix)

plt.show()